In [9]:
import torch
import torchvision
import data_loader

In [10]:
resnet = torchvision.models.resnet18(pretrained=True)

In [11]:
resnet.fc = torch.nn.Linear(in_features=512, out_features=10)

In [12]:
resnet = resnet.to("cuda:0")

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetPartial(nn.Module):
    def __init__(self, original_resnet, num_layers_to_keep):
        super(ResNetPartial, self).__init__()
        self.num_layers_to_keep = num_layers_to_keep
        self.partial_resnet = nn.Sequential(*list(original_resnet.children())[:num_layers_to_keep])

    def forward(self, x):
        return self.partial_resnet(x)

# Load the original ResNet model
resnet = models.resnet18(pretrained=True)

# Number of layers to keep in the partial ResNet
num_layers_to_keep = 6

# Create the partial ResNet model with the desired number of layers
partial_resnet = ResNetPartial(resnet, num_layers_to_keep)

# Move the partial ResNet model to the desired device
partial_resnet = partial_resnet.to("cuda:0")

# Create a random input tensor
X = torch.randn(1, 3, 224, 224).to("cuda:0")

# Forward pass through the partial ResNet
output = partial_resnet(X)
print("Output shape:", output.shape)


In [ ]:
partial_resnet

In [13]:
train_iter, test_iter = data_loader.load_data_cifar_10(batch_size=256, resize=224)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
# 定义损失函数和优化器
criteria = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-4)

In [15]:
# 测试模型
def test():
    resnet.eval()
    eval_loss = 0
    eval_acc = 0
    for inputs, labels in test_iter:
        inputs = inputs.to("cuda:0")
        labels = labels.to("cuda:0")
        outputs = resnet(inputs)
        loss = criteria(outputs, labels)
        eval_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        eval_acc += (preds == labels).float().mean()
    return eval_loss / len(test_iter), eval_acc / len(test_iter)

In [16]:
# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    runnning_loss = 0.0
    running_acc = 0.0
    for inputs, labels in train_iter:
        inputs = inputs.to("cuda:0")
        labels = labels.to("cuda:0")
        outputs = resnet(inputs)
        loss = criteria(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        runnning_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        running_acc += (preds == labels).float().mean()
    eval_loss, eval_acc = test()
    print("epoch %d, loss %.4f, train acc %.3f, test loss %.4f, test acc %.3f" % (epoch + 1, runnning_loss / len(train_iter), running_acc / len(train_iter), eval_loss, eval_acc))
    torch.save(resnet.state_dict(), f"resnet18_epoch{epoch + 1}.pth")

print("training finished")

epoch 1, loss 0.4090, train acc 0.869, test loss 0.2009, test acc 0.933
epoch 2, loss 0.1621, train acc 0.944, test loss 0.2056, test acc 0.930
epoch 3, loss 0.0795, train acc 0.973, test loss 0.1860, test acc 0.939
epoch 4, loss 0.0472, train acc 0.984, test loss 0.2139, test acc 0.937
epoch 5, loss 0.0376, train acc 0.987, test loss 0.2631, test acc 0.931
epoch 6, loss 0.0209, train acc 0.993, test loss 0.2091, test acc 0.944
epoch 7, loss 0.0186, train acc 0.994, test loss 0.2721, test acc 0.932
epoch 8, loss 0.0227, train acc 0.992, test loss 0.2292, test acc 0.940
epoch 9, loss 0.0231, train acc 0.992, test loss 0.2808, test acc 0.938
epoch 10, loss 0.0131, train acc 0.996, test loss 0.2575, test acc 0.942
training finished
